In [ ]:
pip install SimpleITK
pip install dltk
pip install nipype

In [ ]:
from nipype.interfaces import fsl
from nipype.testing import example_data
import nibabel as nib
import numpy as np

import SimpleITK as sitk
import os
import pandas as pd

from matplotlib import pyplot as plt

from dltk.io.augmentation import *
from dltk.io.preprocessing import *

%matplotlib inline

import skimage
from skimage import data
from skimage.transform import resize

# Generate CSV File

In [ ]:
# If there is no a csv file for the data, it can be created by using the code below. Data should be in folders according to their labels.

In [ ]:
#Files should be separated according to their label. Filenames must be the same as label.
def get_images(dir_name=r'./test or train'):
    

    img_names=[]
    
    for label_name in os.listdir(dir_name):
        for img_file_name in os.listdir('/'.join([dir_name, label_name])):
            img_names.append([img_file_name, label_name])

    return img_names

In [ ]:
img_names= get_images("./test")
#Create dataframe and csv
df = pd.DataFrame(img_names, columns=['img_file_name', 'label_name']) # create a dataframe from match list
df.to_csv("test.csv", index=False) # create csv from df

In [ ]:
img_names= get_images("./train")
#Create dataframe and csv
df = pd.DataFrame(img_names, columns=['img_file_name', 'label_name']) # create a dataframe from match list
df.to_csv("train.csv", index=False) # create csv from df

# Prepare The Data

In [ ]:
import gzip
import shutil

#Unzip the FSL zip files

for filename in os.listdir('./skull_stripped_test'): 
  with gzip.open(os.path.join('./skull_stripped_test', filename), 'rb') as f_in:
      with open(os.path.join('./test_unzipped_preprocessed', filename.replace(".gz","")), 'wb') as f_out:
         shutil.copyfileobj(f_in, f_out)

In [ ]:
for filename in os.listdir('./skull_stripped_train'): 
  with gzip.open(os.path.join('./skull_stripped_train', filename), 'rb') as f_in:
      with open(os.path.join('./train_unzipped_preprocessed', filename.replace(".gz","")), 'wb') as f_out:
         shutil.copyfileobj(f_in, f_out)

In [ ]:
#For The Training Data

# Prepare and organize the images
path='./train_unzipped_preprocessed'
for filename in os.listdir(path):   
        complete_file_path = os.path.join(path, filename)  # filename
        # load sitk image
        sitk_moving = sitk.ReadImage(complete_file_path)
        
        t1 = sitk.GetArrayFromImage(sitk_moving)
        # Resizing image to [128, 256, 256] required padding
        #t1_padded = resize_image_with_crop_or_pad(t1, [128, 128, 128], mode='symmetric')
        sitk_moving=skimage.transform.resize(t1, [128, 128, 128])
        t1_norm_zo = normalise_zero_one(sitk_moving) #Diğer koddan alındı
        
        # Crop to [64, 64, 64]
        #t1_cropped = resize_image_with_crop_or_pad(t1, [64, 64, 64], mode='symmetric')


        # Add a feature dimension and normalise
        #t1_norm = np.expand_dims(normalise_one_one(t1_slice), axis=-1)

        # Randomly flip the image along axis 1
        #t1_flipped = flip(t1_norm_zo.copy(), axis=1)

        # Add a Gaussian offset (independently for each channel)
        #t1_offset = add_gaussian_offset(t1_norm_zo.copy(), sigma=0.5)

        # Add Gaussian noise
        #t1_noise = add_gaussian_noise(t1_norm_zo.copy(), sigma=0.25)

        # Elastic transforms. Alpha and sigma parameters 
        #t1_trans_low_s = elastic_transform(t1_norm.copy(), alpha=[1, 1e5, 1e5], sigma=[1, 10, 10])
        #t1_trans_high_s = elastic_transform(t1_norm.copy(), alpha=[10, 2e6, 2e6], sigma=[1, 25, 25])


        new_image=sitk.GetImageFromArray(t1_norm_zo)
        
        # prepare the destination path
        complete_new_path = os.path.join('./train_dataset_preprocessed', 
                                        filename)  #filename
        sitk.WriteImage(new_image, complete_new_path)

In [ ]:
#For The Test Data


# Prepare and organize the images
path='./test_unzipped_preprocessed'
#for files in os.walk(DATABASE):
for filename in os.listdir(path):   
    #for filename in os.listdir(os.path.join(path, classname)):
        complete_file_path = os.path.join(path, filename)  # filename
        # load sitk image
        sitk_moving = sitk.ReadImage(complete_file_path)
        
        # Resizing image to [128, 256, 256] required padding
        
        t1 = sitk.GetArrayFromImage(sitk_moving)
        #t1_padded = resize_image_with_crop_or_pad(t1, [128, 128, 128], mode='symmetric')

        sitk_moving=skimage.transform.resize(t1, [128, 128, 128])
        t1_norm_zo = normalise_zero_one(sitk_moving) #Diğer koddan alındı
        
        # Crop function to [64, 64, 64] can be changed
        #t1_cropped = resize_image_with_crop_or_pad(t1, [64, 64, 64], mode='symmetric')


        # Add a feature dimension and normalise
        #t1_norm = np.expand_dims(normalise_one_one(t1_slice), axis=-1)

        # Randomly flip the image along axis 1
        #t1_flipped = flip(t1_norm_zo.copy(), axis=1)

        # Add a Gaussian offset (independently for each channel)
        #t1_offset = add_gaussian_offset(t1_norm_zo.copy(), sigma=0.5)

        # Add Gaussian noise
        #t1_noise = add_gaussian_noise(t1_norm_zo.copy(), sigma=0.25)

        # Elastic transforms. Alpha and sigma parameters 
        #t1_trans_low_s = elastic_transform(t1_norm.copy(), alpha=[1, 1e5, 1e5], sigma=[1, 10, 10])
        #t1_trans_high_s = elastic_transform(t1_norm.copy(), alpha=[10, 2e6, 2e6], sigma=[1, 25, 25])


        new_image=sitk.GetImageFromArray(t1_norm_zo)


        # Prepare the destination path
        complete_new_path = os.path.join('./test_dataset_preprocessed', 
                                        filename)  #filename
        sitk.WriteImage(new_image, complete_new_path)


In [ ]:
rootdir = r'./train_dataset_preprocessed'
str = "_brain"
for filename in os.listdir(rootdir):
    if str in filename:    
        filepath = os.path.join(rootdir, filename)
        newfilepath = os.path.join(rootdir, filename.replace(str, ""))
        os.rename(filepath, newfilepath)

In [ ]:
rootdir = r'./test_dataset_preprocessed'
str = "_brain"
for filename in os.listdir(rootdir):
    if str in filename:    
        filepath = os.path.join(rootdir, filename)
        newfilepath = os.path.join(rootdir, filename.replace(str, ""))
        os.rename(filepath, newfilepath)

# Training Part

In [ ]:
pip install monai
pip install torchinfo

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import logging
import os
import sys

import numpy as np


import monai
from monai.data import ImageDataset
from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, EnsureType


In [ ]:
from tensorflow import summary
%load_ext tensorboard

In [ ]:
label_to_class = {
    'NC': 0,
    'MCI': 1,
    'AD': 2,
    #Extra classes can be added
    #'NewClass':3
    
}
class_to_label = {v: k for k, v in label_to_class.items()}
n_classes = len(label_to_class)


In [ ]:
monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

images = []
labels=[]
train_image_count=160   #How many images are in the training dataset?

image_labels= pd.read_csv("./train.csv")
image_labels = image_labels.sample(frac=1).reset_index(drop=True)

for index in range(train_image_count-1):
  labels.append(label_to_class[image_labels.iloc[index, 1]])
  images.append(os.path.join("./train_dataset_preprocessed",image_labels.iloc[index,0])) 

labels = np.array(labels)
image_file_names=image_labels.iloc[index,0]

In [ ]:
print(labels)
labels.shape

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
num_epochs=50
#split için değişken oluştur

# Define Training and Validation transforms
train_transforms = Compose([ScaleIntensity(), AddChannel(), RandRotate90(), EnsureType()])  
valid_transforms = Compose([ScaleIntensity(), AddChannel(),  EnsureType()])   

# Define image dataset, data loader
check_ds = ImageDataset(image_files=images, labels=labels,transform=train_transforms)
check_dataloader = DataLoader(check_ds, batch_size=2, number_of_workers=2, pin_memory=torch.cuda.is_available())
im, label = monai.utils.misc.first(check_dataloader)
print(type(im), im.shape, label)

# Create a training data loader. The number of images to be used should be written.
train_dataset= ImageDataset(image_files=images[:144], labels=labels[:144], transform=train_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=7, shuffle=True, number_of_workers=2, pin_memory=torch.cuda.is_available())

# create a validation data loader. The number of images to be used should be written.
valid_dataset = ImageDataset(image_files=images[-15:], labels=labels[-15:], transform=valid_transforms)
valid_dataloader = DataLoader(valid_dataset, batch_size=5, shuffle=True, number_of_workers=2, pin_memory=torch.cuda.is_available())

# Create DenseNet121, CrossEntropyLoss and Adam optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alz_model = monai.networks.nets.DenseNet121(spatial_dims=3,in_channels=1, out_channels=3 ).to(device)  
loss_function = torch.nn.CrossEntropyLoss()
#Starting learning rate is 1e-7 default
optimizer = torch.optim.Adam(alz_model.parameters(),lr=1e-7 ) 
#Max learning rate can be changed according to data
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-5 , steps_per_epoch=len(train_dataloader), epochs=num_epochs)

valid_interval = 2
best_metric = -1
epoch_loss_list = list()
metric_values = list()
writer = SummaryWriter()

In [ ]:
from torchinfo import summary

summary(alz_model)
batch_size = 7

In [ ]:
%tensorboard --logdir /content/runs/RunFileMustBeAdded

In [ ]:
validation_model_training_loss_ls = []
validation_model_training_accuracy_ls = []

for current_epoch in range(num_epochs):
      print("-" * 10)
      print(f"epoch {current_epoch + 1}/{num_epochs}")
      alz_model.train()
      epoch_loss = 0
      epoch_valid_loss=0
      train_step = 0
      valid_step=0
      for batch_data in train_dataloader:
          train_step += 1
          inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
          outputs = alz_model(inputs)
          loss = loss_function(outputs, labels)
          loss.backward()
          optimizer.zero_grad()
          optimizer.train_step()
          scheduler.train_step()
          curr_lr = optimizer.param_groups[0]["lr"]
          epoch_loss += loss.item()
          epoch_len = len(train_dataset) // train_dataloader.batch_size
          print(f"{train_step}/{epoch_len}, train_loss: {loss.item():.4f}")
          writer.add_scalar("train_loss", loss.item(), epoch_len * current_epoch + train_step)
          writer.add_scalar("learning_rate",  curr_lr, epoch_len * current_epoch + train_step)
      epoch_loss /= train_step
      epoch_loss_list.append(epoch_loss)
      print(f"epoch {current_epoch + 1} average loss: {epoch_loss:.4f}")
      #writer.add_scalar("total_train_loss", epoch_loss, epoch_len * current_epoch + train_step)

      if (current_epoch + 1) % valid_interval == 0:
          alz_model.eval()
          with torch.no_grad():
              num_of_correct = 0.0
              metric_count = 0
              for valid_data in valid_dataloader:
                  valid_step=valid_step+1
                  valid_images, valid_labels = valid_data[0].to(device), valid_data[1].to(device)
                  valid_outputs = alz_model(valid_images)
                  value = torch.eq(valid_outputs.argmax(dim=1), valid_labels)
                  metric_count += len(value)
                  num_of_correct += value.sum().item()
                  
                  validation_loss_local_train_fn = loss_function(valid_outputs, valid_labels)
                  epoch_valid_loss += validation_loss_local_train_fn.item()
                  print(f"{train_step}/{epoch_len}, valid_loss: {validation_loss_local_train_fn.item():.4f}")
                  writer.add_scalar("valid_loss", validation_loss_local_train_fn.item(), epoch_len * current_epoch + train_step)
              metric = num_of_correct / metric_count
              metric_values.append(metric)
              epoch_valid_loss /= valid_step
              epoch_loss_list.append(epoch_valid_loss)
              print(f"epoch {current_epoch + 1} average validation loss: {epoch_valid_loss:.4f}")
              
              if metric > best_metric:
                  best_metric = metric
                  best_metric_epoch = current_epoch + 1
                  torch.save(alz_model.state_dict(), "./best_metric_model_classification_3d_array.pth")
                  print("New best metric model is saved")
              print(
                  "current epoch: {} current accuracy: {:.4f} best accuracy: {:.4f} at epoch {}".format(
                      current_epoch + 1, metric, best_metric, best_metric_epoch
                  )
              )
              writer.add_scalar("valid_accuracy", metric, current_epoch + 1)
              
print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
writer.close()

# Evaluation Part

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader

import logging
import os
import sys

import monai
from monai.data import CSVSaver, ImageDataset
from monai.transforms import AddChannel, Compose, Resize, ScaleIntensity, EnsureType

In [ ]:
monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)


images = []
labels = []
test_image_count=40 #How many images are in the test dataset
image_labels= pd.read_csv("./test.csv")

for index in range(test_image_count-1):
  labels.append(label_to_class[image_labels.iloc[index, 1]])
  images.append(os.path.join("./test_dataset_preprocessed",
                                    image_labels.iloc[index,0]))  

labels = np.array(labels)
# Define transforms for images
valid_transforms = Compose([ScaleIntensity(), AddChannel(),  EnsureType()]) 

# Validation image dataset
valid_dataset = ImageDataset(image_files=images, labels=labels, transform=valid_transforms, image_only=False)
# Validation data loader for validation dataset
valid_dataloader = DataLoader(valid_dataset, batch_size=5, number_of_workers=4, pin_memory=torch.cuda.is_available())

# Creating a DenseNet121 architecture based model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alz_model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=3).to(device) #Out channels should be equal to class number

alz_model.load_state_dict(torch.load("./best_metric_model_classification_3d_array.pth"))
alz_model.eval()
with torch.no_grad():
    num_of_correct = 0.0
    metric_count = 0
    csv_saver = CSVSaver(output_dir="./output")
    for valid_data in valid_dataloader:
        valid_images, valid_labels = valid_data[0].to(device), valid_data[1].to(device)
        valid_outputs = alz_model(valid_images).argmax(dim=1)
        value = torch.eq(valid_outputs, valid_labels)
        metric_count += len(value)
        num_of_correct += value.sum().item()
        csv_saver.save_batch(valid_outputs, valid_data[2])
    metric = num_of_correct / metric_count
    print("Evaluation Metric:", metric)
    print("Number of Corrects:",num_of_correct)
    csv_saver.finalize()